In [ ]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#                     .appName("Elasticsearchr") \
#                     .config("spark.executor.memory", "1G") \
#                     .config("spark.driver.memory", "1G") \
#                     .config("spark.driver.maxResultSize", "1G") \
#                     .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.17.23") \
#                     .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
#                     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#                     .getOrCreate()

spark = SparkSession.builder \
                    .appName("Elasticsearchr") \
                    .config("spark.executor.memory", "1G") \
                    .config("spark.driver.memory", "1G") \
                    .config("spark.driver.maxResultSize", "1G") \
                    .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-30_2.12:7.17.23") \
                    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
                    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
                    .config("spark.es.nodes", "elasticsearch") \
                    .config("spark.es.port", "9200") \
                    .config("spark.es.nodes.wan.only", "true") \
                    .config("spark.es.net.ssl", "false") \
                    .config("spark.es.index.auto.create", "true") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark

In [2]:
tuple_list_data = [('Alice', 1), ('Braga', 2), ('Steve', 3)]
df = spark.createDataFrame(tuple_list_data, ['name', 'id'])
df.show()

+-----+---+
| name| id|
+-----+---+
|Alice|  1|
|Braga|  2|
|Steve|  3|
+-----+---+



In [3]:
index_name = "example"

## Passing ES config at read and write

In [7]:
df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "false") \
    .option("es.nodes", "elasticsearch") \
    .option("es.port", 9200) \
    .option("es.index.auto.create", "true") \
    .option("es.resource", f"{index_name}/_doc") \
    .mode("overwrite") \
    .save()

In [8]:
spark.read.format("es") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "false") \
    .option("es.nodes", "elasticsearch") \
    .option("es.port", 9200) \
    .option("es.index.auto.create", "true") \
    .load(index_name) \
    .toPandas()

,id,name
0,2,Braga
1,3,Steve
2,1,Alice


## Using "default" ES config at read and write

In [4]:
df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.resource", f"{index_name}/_doc") \
    .mode("overwrite") \
    .save()

In [5]:
spark.read.format("es").load(index_name).toPandas()

,id,name
0,1,Alice
1,2,Braga
2,3,Steve
